# Import Statements

In [ ]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import random
from math import log
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy
from abc import ABC, abstractmethod
import mpmath as mp
import collections
import time
from statistics import fmean as mean

# Constants

## Debug Level Flags

In [ ]:
DEBUG_LEVEL_NONE = 0
DEBUG_LEVEL_THEORY_1 = 1.1
DEBUG_LEVEL_THEORY_2 = 1.2
DEBUG_LEVEL_FULL = 2
DEBUG_LEVEL_EXTREME = 3

## General Constants

In [ ]:
DEBUG_LEVEL = DEBUG_LEVEL_THEORY_1
NUM_NODES = 100
NUM_EDGES = 150
DEGREE = 3
NUM_MULTICAST_REQUESTS = 20
MAX_MULTICAST_SIZE = NUM_NODES//5
TOLERANCE = 0.3

NATURAL = 0
UNIFORM = 1
PERTURB = 2

# Helper Functions

In [ ]:
def cost(G, prices):
    original_weights = nx.get_edge_attributes(G, "weight") 
    nx.set_edge_attributes(G, prices, "weight")
    retval = sum(nx.get_edge_attributes(G, "weight").values())
    nx.set_edge_attributes(G, original_weights, "weight")
    return retval

def dict_innerProd(x, y):
    assert x.keys() == y.keys()
    return sum(x[key]*y[key] for key in x.keys())

class FrozenDict(collections.abc.Mapping):
    """Don't forget the docstrings!!"""

    def __init__(self, *args, **kwargs):
        self._d = dict(*args, **kwargs)

    def __iter__(self):
        return iter(self._d)

    def __len__(self):
        return len(self._d)

    def __getitem__(self, key):
        return self._d[key]

    def __hash__(self):
        return hash(frozenset(self._d))

# Generating Problem Instances

In [ ]:
def is_connected(G):
    return nx.is_k_edge_connected(G, 1)

def get_random_connected_graph():
    G = nx.empty_graph(NUM_NODES)
    while not is_connected(G):
        G = nx.random_regular_graph(DEGREE, NUM_NODES)
    return G

class MulticastRequest:
    def __init__(self, size, graph, source=None, recipients=None):
        if source is None and recipients is None:
            multicast_group = random.sample(list(graph), size)
            source = multicast_group[0]
            recipients = set(multicast_group[1:])
        self.source = source
        self.recipients = recipients
        self.size = len(recipients) + 1
    
    def multicast_group(self):
        return self.recipients.union([self.source])

class MulticastPackingInstance:
    def __init__(self, num_requests, max_request_size, graph=None, requests=None):
        if graph is None:
            graph = get_random_connected_graph()
        if requests is None:
            requests = list()
            for i in range(num_requests):
                k = random.randint(2, max_request_size)
                requests.append(MulticastRequest(k, graph))
        self.graph = graph
        self.num_edges = len(self.graph.edges())
        self.requests = requests
        self.num_requests = len(requests)

# Create (Reduced) LP

In [ ]:
def create_LP(G, multicast_requests):
    nx.set_edge_attributes(G, 1, "weight")
    reduced_LP = gp.Model("Multicast Packing Model - Reduced")
    congestion = reduced_LP.addVar(name="lambda")
    reduced_LP.setObjective(congestion, gp.GRB.MINIMIZE)
    reduced_LP.update()
    # Packing Constraints
    for e in G.edges():
        reduced_LP.addConstr(congestion >= 0, name="{} congestion".format(sorted(e)))
    # Simplex Constraints
    for i in range(len(multicast_requests)):
        reduced_LP.addConstr(0*congestion == 1, name="Tree Selection for {}".format(i))
    reduced_LP.update()
    return reduced_LP

# Column Generating Subproblem

## Abstract Column Generator

In [ ]:
class MulticastPackingColumnGenerator(ABC):
    def __init__(self, instance, reduced_LP):
        self.instance = instance
        self.reduced_LP = reduced_LP
        self.Gurobi_variables = [dict() for i in range(self.instance.num_requests)]
        
    @abstractmethod
    def generate_tree(self, i, prices):
        pass
    
    def generate_new_trees(self, prices=1):
        new_trees = list()
        assert nx.is_weighted(self.instance.graph)
        for i in range(self.instance.num_requests):
            new_tree = self.generate_tree(i, prices)
            new_trees.append(new_tree)
            
            coeffs = list()
            constrs = list()
            # the new tree contributes to the constraint for each of its edges
            for e in new_tree.edges():
                coeffs.append(-1)
                constrs.append(self.reduced_LP.getConstrByName(
                    "{} congestion".format(sorted(e))))
            
            # the new tree contributes to the selection constraint 
            # for the corresponding multicast request
            coeffs.append(1)
            constrs.append(self.reduced_LP.getConstrByName(
                "Tree Selection for {}".format(i)))
            self.Gurobi_variables[i][new_tree] = self.reduced_LP.addVar(
                name="x({}, {})".format(i, nx.info(new_tree)), 
                column=gp.Column(coeffs, constrs))
            self.reduced_LP.update()
            
        return new_trees

## Simple 2-Approximate Steiner Tree Column Generator

In [ ]:
class Approx2MulticastPackingColumnGenerator(MulticastPackingColumnGenerator):
    def generate_tree(self, i, prices):
        original_weights = nx.get_edge_attributes(self.instance.graph, "weight") 
        nx.set_edge_attributes(self.instance.graph, prices, "weight")
        retval = nx.algorithms.approximation.steinertree.steiner_tree(
            self.instance.graph, self.instance.requests[i].multicast_group())
        nx.set_edge_attributes(self.instance.graph, original_weights, "weight")
        return retval

## Steiner Tree IP Solving Column Generator

In [ ]:
class ExactMulticastPackingColumnGeneratorIP(MulticastPackingColumnGenerator):
    def __init__(self, instance, reduced_LP):
        super().__init__(instance, reduced_LP)
        self.GurobiModels = list()
        self.variables = list()
        D = nx.DiGraph(instance.graph)
        for i in range(instance.num_requests):
            source = instance.requests[i].source
            recipients = instance.requests[i].recipients
            
            model = gp.Model("Steiner Tree IP (MCF) for Request {}".format(i))
            # Add variables
            x = {e : model.addVar(vtype=gp.GRB.BINARY, 
                                  name="{} selection".format(e)) 
                     for e in D.edges()}
            f = {r : {e: model.addVar(vtype=gp.GRB.BINARY, 
                                      name="{} flow to {}".format(e,r)) 
                          for e in D.edges()} 
                     for r in recipients}
            
            # Add constraints
            for r in recipients:
                for v in D.nodes():
                    inflow = sum(f[r][(u,v)] for u in D.predecessors(v))
                    outflow = sum(f[r][(v,w)] for w in D.successors(v))
                    if v == source:
                        netflow = -1
                    elif v == r:
                        netflow = 1
                    else:
                        netflow = 0
                    
                    model.addConstr(inflow - outflow == netflow,
                                    name="{}-flow conservation for {}".format(r,v))
                
                for e in D.edges():
                    model.addConstr(f[r][e] <= x[e],
                                    name="{} availability for {}-flow".format(e,r))
            
            # Note we don't add the objective as that is determined by
            # the prices in each iteration.
            model.update()
            
            self.variables.append((x,f))
            self.GurobiModels.append(model)
    
    
    def generate_tree(self, i, prices):
        model = self.GurobiModels[i]
        x = self.variables[i][0]
        c = dict()
        
        if isinstance(prices, dict):
            for e in x:
                if e in prices:
                    c[e] = prices[e]
                elif tuple(reversed(e)) in prices:
                    c[e] = prices[tuple(reversed(e))]
                else:
                    raise KeyError('Edge not in prices?!')
        else: # should be a constant numeric value
            for e in x:
                c[e] = prices
                    
        
        model.setObjective(sum(c[e]*x[e] for e in x), gp.GRB.MINIMIZE)
        model.update()
        model.optimize()
        
        steinerTree = list()
        for e in self.instance.graph.edges():
            if (x[e].X == 1) or (x[tuple(reversed(e))].X == 1):
                steinerTree.append(e)
        
        return self.instance.graph.edge_subgraph(steinerTree)

# Solvers

## Abstract Solver Class

In [ ]:
class MulticastPackingSolver(ABC):
    
    # Constructor
    def __init__(self, instance=None, block_solver=None):
        # Attributes related to problem instance
        if instance is None:
            instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
        self.instance = instance
        self.reduced_LP = create_LP(self.instance.graph, self.instance.requests)
        
        # Attributes related to how this solver generates columns and solutions
        if block_solver == None:
            block_solver = Approx2MulticastPackingColumnGenerator(self.instance, self.reduced_LP)
        if block_solver == "Exact":
            block_solver = ExactMulticastPackingColumnGeneratorIP(self.instance, self.reduced_LP)
        self.column_generator = block_solver
        
        # Attributes that track things
        self.tol = TOLERANCE
        self.t = mp.mpf(0)
        self.stop_flag = False
        self.iteration = 0
        self.solution = list()
        self.objVal = dict()
        self.fVal = dict()
        self.price = dict()
        self.multicast_costs = dict()
        self.new_trees = self.column_generator.generate_new_trees()
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
            print(reduced_LP.getA().toarray())
            print(reduced_LP.getAttr("RHS"))
    
    # Abstract methods for generating values of functions needed by the solver
    
    @abstractmethod
    def get_next_solution(self):
        pass
    
    @abstractmethod
    def perform_checks_and_updates(self):
        pass
    
    @abstractmethod
    def generate_lamb(self, x):
        pass
    
    @abstractmethod
    def generate_fVal(self, x):
        pass
     
    @abstractmethod   
    def generate_p(self, x, t):
        pass
    
    @abstractmethod
    def generate_q(self, x, t):
        pass
    
    # Methods for accessing values of functions needed by the solver
    
    def lamb(self, x):
        if x not in self.objVal:
            self.generate_lamb(x)
        return self.objVal[x]
    
    def f(self, x):
        if x not in self.fVal:
            self.generate_fVal(x)
        return self.fVal[x]
    
    def p(self, x, t=0):
        if (x,t) not in self.price:
            self.generate_p(x,t)
        return self.price[(x,t)]
    
    def q(self, x, t=0):
        if (x,t) not in self.multicast_costs:
            self.generate_q(x,t)
        return self.multicast_costs[(x,t)]
    
    def theta(self, x, t=0):
        return self.lamb(x)
    
    def phi(self, x, t=0):
        return log(self.lamb(x))
    
    def Phi(self, theta, x, t=0):
        return 
    
    def toleranceFunction(self, offset=0):
        x = self.solution[self.iteration-offset]
        f_prime = dict()
        for e in self.instance.graph.edges():
            f_prime[tuple(sorted(e))] = mp.mpf(0)
            for i in range(self.instance.num_requests):
                if self.new_trees[i].has_edge(*e):
                    f_prime[tuple(sorted(e))] += 1
    
        pf = dict_innerProd(self.p(x, self.t), self.f(x))
        pf_prime = dict_innerProd(self.p(x, self.t), f_prime)
        return (pf-pf_prime)/(pf+pf_prime)
    
    # Main Function
    def perform_iteration(self):
        t = self.t
        x = self.get_next_solution()
        self.solution.append(x)
        self.new_trees = self.column_generator.generate_new_trees(self.p(x, t))
        self.perform_checks_and_updates(x)
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_2:
            print(self.iteration)
            print("lambda(x): {}".format(self.lamb(x)))
            print("phi_t(x): {}".format(self.phi(x,t)))
            print("tolerance: {}".format(self.toleranceFunction()))
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_1 and (self.iteration % 100 == 0 or self.stop_flag):
            print(self.iteration)
            print("lambda(x): {}".format(self.lamb(x)))
            print("phi_t(x): {}".format(self.phi(x,t)))
            print("tolerance: {}".format(self.toleranceFunction()))
            
        self.iteration += 1

## Pure Column Generation Solver Class

In [ ]:
class PureColGenMcpSolver(MulticastPackingSolver):
    def get_next_solution(self):
        self.reduced_LP.optimize()
        x = [dict() for i in range(self.instance.num_requests)]
        for i in range(self.instance.num_requests):
            for T in self.column_generator.Gurobi_variables[i]:
                value = self.column_generator.Gurobi_variables[i][T].getAttr(gp.GRB.Attr.X)
                if 1 >= value > 0: # Should have better check that var isn't lambda
                    x[i][T] = value
            x[i] = FrozenDict(x[i])
        
        x = tuple(x)
        self.generate_lamb(x)
        self.generate_fVal(x)
        self.generate_p(x)
        self.generate_q(x)
        
        return x
    
    def perform_checks_and_updates(self, x):
        if ( (sum([cost(self.new_trees[i], self.p(x)) for i in range(self.instance.num_requests)]) >= self.lamb(x))
            or (all([cost(self.new_trees[i], self.p(x)) - self.q(x)[i] >= 0 for i in range(self.instance.num_requests)]))
            or (self.toleranceFunction() <= self.tol) ):
                self.stop_flag = True
                # Could have different stop flags for different stopping criteria
    
    def generate_lamb(self, x):
        self.objVal[x] = self.reduced_LP.getObjective().getValue()
        # Doesn't work if x is not current LP solution
    
    def generate_fVal(self, x):
        self.fVal[x] = dict()
        for e in self.instance.graph.edges():
            self.fVal[x][tuple(sorted(e))] = self.lamb(x) + self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack) # Note: Gurobi signs their slacks stupidly
        
    def generate_p(self, x, t=0):
        self.price[(x,t)] = dict()
        for e in self.instance.graph.edges():
            self.price[(x,t)][tuple(sorted(e))] = self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Pi)
            
    def generate_q(self, x, t=0):
        self.multicast_costs[(x,t)] = [None] * len(self.instance.requests)
        for i in range(self.instance.num_requests):
            self.multicast_costs[(x,t)][i] = self.reduced_LP.getConstrByName("Tree Selection for {}".format(i)).getAttr(gp.GRB.Attr.Pi)

## Jansen Zhang 2008 Convex MinMax Solver

In [ ]:
def theta_eq(theta, t, M, f):
    retval = mp.mpf(0)
    
    if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
        print("retval: {}".format(type(retval)))
        print("theta: {}: {}".format(type(theta), theta))
    
    for e in f:
        retval += theta/(theta - f[tuple(sorted(e))])
    retval = t*retval/M - 1

    if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
        print("theta = {}".format(theta))
        print("theta_eq = {}".format(retval))
        

    return retval

def derivative_theta_eq(theta, t, M, f):
    retval = mp.mpf(0)
    for e in f:
        retval += f[tuple(sorted(e))]/((theta - f[tuple(sorted(e))])*(theta - f[tuple(sorted(e))]))
    retval = -t*retval/M

    if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
        print("d_theta_eq = {}".format(retval))

    return retval
    
    

class JansenZhangMinMaxer(MulticastPackingSolver):
    
    def __init__(self, instance=None, block_solver=None, sigma0=1):
        super().__init__(instance, block_solver)
        self.sigma = mp.mpf(sigma0)
        self.t = self.sigma/6
        self.M = mp.mpf(len(self.instance.graph.edges()))
        self.theta_dict = dict()
        self.phi_dict = dict()
        self.finished_coordination = False
        self.lambda_of_prev_scaling = 0
        self.w = None
        
        
    def get_next_solution(self):
        if self.iteration == 0:
            x = [dict() for i in range(self.instance.num_requests)]
            step_size = mp.mpf(1)
        else:
            x = self.solution[self.iteration-1]
            
            f_prime = dict()
            for e in self.instance.graph.edges():
                f_prime[tuple(sorted(e))] = mp.mpf(0)
                for i in range(self.instance.num_requests):
                    if self.new_trees[i].has_edge(*e):
                        f_prime[tuple(sorted(e))] += mp.mpf(1)
    
            pf = dict_innerProd(self.p(x, self.t), self.f(x))
            pf_prime = dict_innerProd(self.p(x, self.t), f_prime)
            step_size = ( (self.t*self.theta(x, self.t)*self.toleranceFunction(1)) 
                   /(2*self.M*(pf+pf_prime)) )

        
        new_x = [dict() for i in range(self.instance.num_requests)]

        for i in range(self.instance.num_requests):
            for T in x[i]:
                new_x[i][T] = (1-step_size)*x[i][T]
                
            if self.new_trees[i] in new_x[i]:
                new_x[i][self.new_trees[i]] += step_size
            else:
                new_x[i][self.new_trees[i]] = step_size
            new_x[i] = FrozenDict(new_x[i])
        return tuple(new_x)
    
    def perform_checks_and_updates(self, x):
        if self.w == None:
            if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_2:
                print("Scaling Phase 0 Over")
            self.sigma = self.sigma/2
            self.t = self.sigma/6
            self.w = mp.mpf((1+self.sigma)/((1+self.sigma/3)*self.M))
            self.new_trees = self.column_generator.generate_new_trees(self.p(x, self.t))
        elif ( (self.toleranceFunction() <= self.sigma/6) 
              or (self.lamb(x) <= self.w*self.lambda_of_prev_scaling) ):
            if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_2:
                print("New Scaling Phase")
            if self.sigma <= self.tol:
                self.stop_flag = True
            else:
                self.lambda_of_prev_scaling= self.lamb(x)
                self.sigma = self.sigma/2
                self.t = self.sigma/6
                self.w = mp.mpf((1+self.sigma)/(1+2*self.sigma))
                self.new_trees = self.column_generator.generate_new_trees(self.p(x, self.t))
                
    
    def generate_lamb(self, x):
        self.objVal[x] = max(self.f(x).values())
    
    def generate_fVal(self, x):
        self.fVal[x] = {tuple(sorted(e)): 0 for e in self.instance.graph.edges()}
        for i in range(self.instance.num_requests):
            for T in x[i]:
                for e in T.edges():
                    self.fVal[x][tuple(sorted(e))] += x[i][T]
        
    def generate_p(self, x, t):
        self.price[(x,t)] = dict()
        for e in self.instance.graph.edges():
            self.price[(x,t)][tuple(sorted(e))] = t*self.theta(x,t)/(self.M*(self.theta(x,t) 
                                                                              - self.f(x)[tuple(sorted(e))]))
    
    # THIS ISN'T NEEDED BY THIS ALGO. I WOULD LIKE TO ADD THIS FUNCTION BUT CURRENTLY THIS WON'T WORK
    def generate_q(self, x, t=0):
        return
        self.multicast_costs[(x,t)] = [None] * len(self.instance.requests)
        for i in range(self.instance.num_requests):
            self.multicast_costs[(x,t)][i] = self.reduced_LP.getConstrByName(
                "Tree Selection for {}".format(i)).getAttr(gp.GRB.Attr.Pi)
            
    def generate_theta(self, x, t):
        if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
            #print(t)
            #print(x)
            #print(self.lamb(x))
            #print(self.f(x))
            print("lambda: {} \t t: {} \t M: {}".format(type(self.lamb(x)), type(t), type(self.M)))
            print("Result: {}".format(type(self.lamb(x)/(1-t/self.M))))
            print(self.lamb(x)/(1-t/self.M))
            #print(theta_eq(self.lamb(x)/(1-t/self.M), t, self.M, self.f(x)))
            print(self.lamb(x)/(1-t))
            #print(theta_eq(self.lamb(x)/(1-t), t, self.M, self.f(x)))
        #self.theta_dict[(x,t)] = scipy.optimize.brentq(
        #    theta_eq, self.lamb(x)/(1-t/self.M), self.lamb(x)/(1-t), args=(t, self.M, self.f(x)))
        self.theta_dict[(x,t)] = (
            mp.findroot(lambda theta: theta_eq(theta, t, self.M, self.f(x)),
                        (mp.mpf(self.lamb(x)/(1-t/self.M)), mp.mpf(self.lamb(x)/(1-t))), 
                        solver='ridder')
        )
        
        
        #self.theta_dict[(x,t)] = scipy.optimize.newton(
        #     theta_eq, self.lamb(x)/(1-t), derivative_theta_eq, args=(t, self.M, self.f(x)))
        
    # THE FOLLOWING IS DEPRECATED AND WILL CAUSE ERRORS
    # Define a function that computes theta_t(x)
    #def theta_via_sympy(self, x, t):
    #    M = self.M
    #    f = self.f
    #    
    #    theta = sympy.Symbol('theta')
    #    expression = 1
    #    for m in constraint_indices:
    #        expression -= (t/M) * theta/(theta - f(x,m))
    #    soln_set = sympy.solvers.solveset(expression, theta, domain=sympy.Interval(self.lamb[x], sympy.S.Infinity, True, True))
    #    #assert len(soln_set == 1)
    #    if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
    #        print(soln_set)
    #    for soln in soln_set:
    #        retval = soln
    #    return retval
            
    def generate_phi(self, x, t):
        theta = self.theta(x,t)
        phi = mp.mpf(0)
        for e in self.instance.graph.edges():
            phi -= log(theta - self.f(x)[tuple(sorted(e))])
        phi = phi*t/self.M
        phi += log(theta)
        self.phi_dict[(x,t)] = phi
    
    def theta(self, x, t):
        if (x,t) not in self.theta_dict:
            self.generate_theta(x,t)
        return self.theta_dict[(x,t)]
    
    def phi(self, x, t):
        if (x,t) not in self.phi_dict:
            self.generate_phi(x,t)
        return self.phi_dict[(x,t)]

# Testing

In [ ]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_connected_graph())
    G = get_random_connected_graph()
    for i in range(num_tests):
        request = MulticastRequest(MAX_MULTICAST_SIZE, G)
        source = request.source
        recipients = request.recipients
        assert request.source is not None
        assert request.recipients
        assert not request.source in request.recipients
        assert isinstance(request.recipients, set)
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    for request in instance.requests:
        assert request.source in instance.graph
        assert request.recipients.issubset(instance.graph)

def test_LP_creation(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    column_generator = Approx2MulticastPackingColumnGenerator(instance, reduced_LP)
    
    new_trees = column_generator.generate_new_trees()
    assert (len(new_trees) == instance.num_requests)
    for tree in new_trees:
        assert cost(tree, 1) == len(tree.edges()), "cost: {} edges: {}".format(cost(tree, 1), len(tree.edges()))
           
    assert len(reduced_LP.getVars()) == 1+instance.num_requests
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 1
run_tests(NUM_TESTS)

# Rounding Algorithms

## (Totally) Randomized Rounding

In [ ]:
def randRoundingHelper(fracSoln):
    randNum = random.random()
    for T in fracSoln:
        randNum -= fracSoln[T]
        if randNum < 0:
            return T
    
    
def randRounding(MCPsolver):
    intSoln = list()
    fracSoln = MCPsolver.solution[MCPsolver.iteration-1]
    for i in range(MCPsolver.instance.num_requests):
        intSoln.append(FrozenDict({randRoundingHelper(fracSoln[i]) : 1}))
        
    return tuple(intSoln)

fracSolnCG = CGsolver.solution[CGsolver.iteration-1]
fracSolnCG2 = CGsolver2.solution[CGsolver2.iteration-1]
#fracSolnJZ = MCPsolver.solution[MCPsolver.iteration-1]
roundedCG = randRounding(CGsolver)
roundedCG2 = randRounding(CGsolver2)
#roundedJZ = randRounding(JZsolver)
print(JZsolver.lamb(roundedCG))
print(JZsolver.lamb(roundedCG2))
#print(JZsolver.lamb(roundedJZ))

# Run the solver

In [ ]:
for i in range(1):
    print("BEGIN NEW TEST")
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    print("Column Generation - 2-approx")
    CGsolver = PureColGenMcpSolver(instance=instance)
    CGtimer = list()
    while(not CGsolver.stop_flag):
        prev_time = time.perf_counter()
        CGsolver.perform_iteration()
        new_time = time.perf_counter()
        CGtimer.append(new_time - prev_time)
    print()
    print("Average Time of Pure Column Generation Iteration with 2-approx block solver: {}".format(mean(CGtimer)))
    print()
    
    print("Column Generation - Exact")
    CGsolver2 = PureColGenMcpSolver(instance=instance, block_solver="Exact")
    CGtimer2 = list()
    while(not CGsolver2.stop_flag):
        prev_time = time.perf_counter()
        CGsolver2.perform_iteration()
        new_time = time.perf_counter()
        CGtimer2.append(new_time - prev_time)
    print()
    print("Average Time of Pure Column Generation Iteration with an exact block solver: {}".format(mean(CGtimer2)))
    print()
    
    # print("Jansen-Zhang Algorithm")
    JZsolver = JansenZhangMinMaxer(instance=instance)
    # JZtimer = list()
    # while(not JZsolver.stop_flag):
    #     prev_time = time.perf_counter()
    #     JZsolver.perform_iteration()
    #     new_time = time.perf_counter()
    #     JZtimer.append(new_time - prev_time)
    # print()
    # print("Average Time of Jansen-Zhang Algorithm Iteration: {}".format(mean(JZtimer)))

In [ ]:
x = fracSolnCG
z = roundedCG
for i in range(len(x)):
    for T in z[i]:
        print(T)
        print(z[i][T])
    for T in x[i]:
        print(T)
        print(x[i][T])
